In [7]:
import os
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, TensorDataset
from PIL import Image
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [16]:

# Constants
BATCH = 16
IM_SIZE = 224
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
NUM_EPOCHS = 25
LEARNING_RATE = 0.01

# Load the data
df = pd.read_csv("train.csv")

# Filter dataset with hotel_ids >= 20
s1 = df["hotel_id"].value_counts()
s2 = s1[s1 >= 20]
hotel_id_list = s2.index.tolist()
df1 = df[df["hotel_id"].isin(hotel_id_list)]

# Train-test split (90:10)
train_df, test_df = train_test_split(df1, test_size=0.1, stratify=df1['hotel_id'], random_state=42)

# Map hotel_id to indices for classification
unique_hotel_ids = sorted(train_df['hotel_id'].unique())
hotel_id_to_index = {hotel_id: idx for idx, hotel_id in enumerate(unique_hotel_ids)}
index_to_hotel_id = {idx: hotel_id for hotel_id, idx in hotel_id_to_index.items()}

# Add the class index to the DataFrame
train_df['class_index'] = train_df['hotel_id'].map(hotel_id_to_index)
test_df['class_index'] = test_df['hotel_id'].map(hotel_id_to_index)


# Define transformations
Transform = transforms.Compose([
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(IM_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Dataset Class
class HotelImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['image']
        label = self.df.iloc[idx]['class_index']
        img_path = os.path.join(self.image_dir, str(self.df.iloc[idx]['chain']), img_name)
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Initialize datasets and loaders
image_dir = "train_images"
train_dataset = HotelImageDataset(train_df, image_dir, Transform)
test_dataset = HotelImageDataset(test_df, image_dir, Transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH, shuffle=False)


model = models.resnet50(pretrained=True)

# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last two stages (Layer3, Layer4) and fc layer
for name, param in model.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name:
        param.requires_grad = True


# Modify the last fully connected layer to match the number of classes
num_classes = len(unique_hotel_ids)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to GPU if available
model = model.to(DEVICE)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE, momentum=0.9)

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{NUM_EPOCHS}"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Calculate average training loss for the epoch
    avg_train_loss = running_loss / len(train_loader)

    # Validation step
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc=f"Validation Epoch {epoch+1}/{NUM_EPOCHS}"):
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    # Print training and validation results for the epoch
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

print("Finished Training the ResNet model.")

# Save the fine-tuned ResNet model
torch.save(model.state_dict(), "data_augmented_resnet2-4on50.pth")


/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Training Epoch 1/25:   0%|          | 9/2160 [00:05<22:46,  1.57it/s]


KeyboardInterrupt: 

In [3]:
print(len(hotel_id_list))

1062


In [11]:
# Validation step
model.eval()  # Set model to evaluation mode
val_loss = 0.0
correct = 0
total = 0
topX_correct = 0  # Initialize top-X correct count
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc=f"Validation Epoch {epoch+1}/{NUM_EPOCHS}"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        # Calculate top-1 accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Calculate top-X accuracy
        _, predicted_topX = outputs.data.topk(TOP_X, dim=1, largest=True, sorted=True)
        labels_expanded = labels.view(-1, 1).expand_as(predicted_topX)
        topX_correct += (predicted_topX == labels_expanded).sum().item()

        # Optional: Collect top-X predictions
        # batch_topX_predictions = predicted_topX.cpu().numpy()
        # Process 'batch_topX_predictions' as needed

avg_val_loss = val_loss / len(test_loader)
val_accuracy = 100 * correct / total
topX_accuracy = 100 * topX_correct / total

# Print training and validation results for the epoch
print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%, Top-{TOP_X} Validation Accuracy: {topX_accuracy:.2f}%")


/var/folders/pr/6qx50l4x5dzg_dp34rn1wg3c0000gn/T/ipykernel_31547/247071599.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('data_augment

TypeError: 'map_locations' is an invalid keyword argument for Unpickler()